In [40]:
import open3d as o3d
print(o3d.__version__)

import numpy as np
import copy
import time

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

0.15.1


In [41]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [42]:
def prepare_dataset(voxel_size):
    print(":: Load two point clouds and disturb initial pose.")

    demo_icp_pcds = o3d.data.DemoICPPointClouds()
    target = o3d.io.read_point_cloud("fotogrametri.ply")
    source = o3d.io.read_point_cloud("lidar-1.ply") 
    # trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
    #                          [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    # source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

In [43]:
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result

In [44]:
def refine_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.4
    print(":: Point-to-plane ICP registration is applied on original point")
    print("   clouds to refine the alignment. This time we use a strict")
    print("   distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_icp(
        source, target, distance_threshold, result_ransac.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    return result


In [45]:
voxel_size = 0.03  # means 5cm for the dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(voxel_size)

# result_ransac = execute_global_registration(source_down, target_down,
# source_fpfh, target_fpfh,
# voxel_size)
# print(result_ransac)
# draw_registration_result(source_down, target_down,result_ransac.transformation)

result_icp = refine_registration(source, target, source_fpfh, target_fpfh,voxel_size)
print(result_icp)
draw_registration_result(source, target, result_icp.transformation)

:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.030.
:: Estimate normal with search radius 0.060.
:: Compute FPFH feature with search radius 0.150.
:: Downsample with a voxel size 0.030.
:: Estimate normal with search radius 0.060.
:: Compute FPFH feature with search radius 0.150.
:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 0.012.
RegistrationResult with fitness=8.476013e-05, inlier_rmse=8.603197e-03, and correspondence_set size of 8
Access transformation to get result.


In [50]:

threshold = 0.009
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, result_icp.transformation,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)


RegistrationResult with fitness=1.059502e-04, inlier_rmse=6.058691e-03, and correspondence_set size of 10
Access transformation to get result.
Transformation is:
[[ 9.99996356e-01  4.19332902e-04  2.66677827e-03  1.74282922e-02]
 [-4.22504097e-04  9.99999204e-01  1.18869630e-03  1.69232037e-02]
 [-2.66627769e-03 -1.18981870e-03  9.99995738e-01  6.83870235e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [51]:

test = copy.deepcopy(target).transform(reg_p2p.transformation)

# #color source to yellow
# source.paint_uniform_color([1, 0.706, 0])
# #color target to blue
# target.paint_uniform_color([0, 0.651, 0.929])
# #color test to red
# test.paint_uniform_color([1, 0, 0])

o3d.visualization.draw_geometries([source])
o3d.visualization.draw_geometries([target])

o3d.visualization.draw_geometries([source,test])

test2 = source+test

o3d.visualization.draw_geometries([test2])

#o3d.visualization.draw_geometries([test])
#o3d.visualization.draw_geometries([source, target, test])


In [52]:
o3d.io.write_point_cloud("export.ply",test2,write_ascii=True)

True